In [0]:
from pathlib import Path
import networkx as nx
from networkx import NetworkXError
import matplotlib.pyplot as plt
import pandas as pd
from random import random

ROOT = Path('drive/My Drive/PWr/ONoS/Task2')

In [0]:
def init_graph(network_name):
    g = nx.Graph()
    network_path = ROOT/'networks'/network_name
    with open(network_path / 'edges.csv', "r") as f:
        g = nx.parse_edgelist(f, nodetype=int, create_using=g)
    try:
        with open(network_path / 'nodes.csv', "r") as f:
            node_labels = map(lambda x: x.rstrip('\n'), f.readlines())
            mapping = {k: v for k, v in enumerate(node_labels, start=1)}
            g = nx.relabel_nodes(g, mapping)
    except IOError:
        print("Unlabeled graph")
    return g

In [0]:
def normalize(x, new_lower_bound, new_upper_bound):
  minim = min(x)
  maxim = max(x)
  old_range = maxim - minim
  new_range = new_upper_bound - new_lower_bound
  return [((a - minim) / old_range) * new_range + new_lower_bound for a in x]

def get_random_pos_attr():
  return random() * 2 - 1

def get_positions(infile,cities):
  df = pd.read_csv(infile)

  latts = list(df["latt"])
  longts = list(df["longt"])
  
  latts = list(map(lambda x: x if x!=0 and x<360 else get_random_pos_attr(),latts))
  longts = list(map(lambda x: x if x!=0 and x<360 else get_random_pos_attr(),longts))

  latts_norm = normalize(latts,-1,1)
  longts_norm = normalize(longts,-1,1)

  positions = list(zip(latts_norm,longts_norm))
  positions_dict = dict(zip(cities,positions))

  return positions_dict

def normalize(x, new_lower_bound, new_upper_bound):
  minim = min(x)
  maxim = max(x)
  old_range = maxim - minim
  new_range = new_upper_bound - new_lower_bound
  return [((a - minim) / old_range) * new_range + new_lower_bound for a in x]
  
def visualize_graph_1():
  g = init_graph('subelj_euroroad')
  pos = get_positions(ROOT/'geodata.csv',list(g.nodes()))

  betweennesses = dict(nx.betweenness_centrality(g)).values()
  node_size = map(lambda x: x * 1000, betweennesses)
  node_size = list(map(lambda x: 5 if x<5 else x, node_size))
  fig, ax = plt.subplots(figsize=(10,10))
  nx.draw_networkx(g,ax=ax,with_labels=False,node_size=node_size,pos=pos)
  plt.title(label="Betwenness (geoposition)")

  fig, ax = plt.subplots(figsize=(10,10))
  nx.draw_spring(g,ax=ax,with_labels=False,node_size=node_size,label="Betwenness")
  plt.title(label="Betwenness (spring layout)")

  degrees = dict(nx.degree(g)).values()
  node_size = list(map(lambda x:x * 10, degrees))
  node_size = list(map(lambda x: 5 if x<5 else x, node_size))
  fig, ax = plt.subplots(figsize=(10,10))
  nx.draw_spring(g,with_labels=False,ax=ax,node_size=node_size,label="Degrees")
  plt.title(label="Degrees (spring layout)")

def visualize_graph_2():
  g = init_graph('ca-AstroPh')

  fig, ax = plt.subplots(figsize=(10,10))
  nx.draw_networkx(g,ax=ax,with_labels=False,node_size=50)

  fig, ax = plt.subplots(figsize=(10,10))
  degrees = dict(nx.degree(g)).values()
  node_size = normalize(degrees,50,200)
  nx.draw_networkx(g,ax=ax,with_labels=False,node_size=node_size)
  plt.title('Degree')

def visualize_graph_3():
  g = init_graph('livemocha')

  fig, ax = plt.subplots(figsize=(10,10))
  nx.draw_networkx(g,ax=ax,with_labels=False,node_size=50)

  fig, ax = plt.subplots(figsize=(10,10))
  degrees = dict(nx.degree(g)).values()
  node_size = normalize(degrees,50,200)
  nx.draw_networkx(g,ax=ax,with_labels=False,node_size=node_size)
  plt.title('Degree')

In [0]:
def all_visualisations():
  print('-------subelj_euroroad--------')
  visualize_graph_1()

  print('-------ca-AstroPh-------')
  visualize_graph_2()

  print('-------livemocha-------')
  visualize_graph_3()

In [0]:
all_visualisations()